In [0]:
import os
from datetime import datetime
from delta.tables import DeltaTable
from global_lib.global_utils.logging import GlobalLogger

In [0]:
catalog = ""
schema = ""
table = ""

logger = GlobalLogger()


In [0]:
# Config
TABLE_NAME = "your_catalog.schema.grants_table"  # Replace with your actual table name
logger = GlobalLogger()

# Delete records where marked_for_deletion = true
delta_table = DeltaTable.forName(spark, TABLE_NAME)
delta_table.delete("marked_for_deletion = true")
print("Deleted rows where marked_for_deletion = true")
# logger.log("Deleted rows where marked_for_deletion = true")

# Process remaining records for expiry
df = spark.table(TABLE_NAME)
current_time = datetime.utcnow()

rows = df.select(
    "id", "system", "environment", "object_type", "object_name",
    "privilege", "principal", "action", "expiry"
).collect()

for row in rows:
    expiry = row["expiry"]

    if expiry is None:
        print(f"Skipping id={row['id']} (no expiry date)")
        continue

    if expiry > current_time:
        print(f"Skipping id={row['id']} (expiry date in the future)")
        continue

    # Expired — revoke directly
    print(f"Revoking expired privilege for principal={row['principal']} on object={row['object_name']}")

    try:
        revoke_stmt = f"""
            REVOKE {row['privilege']} ON {row['object_type']} {row['object_name']} FROM `{row['principal']}`
        """
        spark.sql(revoke_stmt)

    except Exception as e:
        print(f"Failed to revoke for id={row['id']}: {e}")
        logger.log(e, custom_input={"object_name": "ExpirationProcessor"})
